In [27]:
import kfp
import kfp.dsl as dsl
from kfp.components import func_to_container_op
import time


@func_to_container_op
def result_output(result) -> str:
    import json
    result = result.replace("'", r"\"")
    out_dict = json.loads(result)['out']
    metadata_dict = json.loads(out_dict)
    return metadata_dict['name']

def echo_op(text):
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23'
        command=['sh', '-c'],
        arguments=['echo "$0"', text],
    )

@dsl.pipeline(
    name='katib_pipeline',
    description='submit katibe experiemnt on pipeline'
)
def katib_pipeline(host, token, experiment_name):
    
    exit_task = echo_op("done")
    
    with dsl.ExitHandler(exit_task): 
                
        katib = dsl.ContainerOp(
            name='SubmitKatibExperiment',
            image='kubeflow-registry.default.svc.cluster.local:30000/experiement-runner:4B7896D4',
            command=['python', '/app/notebook_submit_experment_with_token.py'],
            arguments=[
                "--host", host,
                "--token", token,
                "--experiment_name", experiment_name
            ],
            file_outputs={'out': '/tmp/result.json'}
        )
        result = result_output(katib.outputs).after(katib)
        echo_op(result.output)


arguments = {'host': 'https://10.x.0.x:6443',
             'experiment_name': 'dudaji-katib',
             'token' : 'eyJhbG...fVQ'
            }
    
if __name__ == '__main__':
    kfp.Client().create_run_from_pipeline_func(pipeline_func=katib_pipeline, 
                                               arguments=arguments)

In [ ]:
!pip install json